In [ ]:
import torch
import torch.utils.cpp_extension
from torch import Tensor, jit, nn

torch.utils.cpp_extension.load(
    name="op_with_autograd",
    sources=["op.cpp"],
    is_python_module=False,
    verbose=True,
)

In [ ]:
op_with_autograd = torch.ops.custom.op_with_autograd

a = torch.randn(3, 3).cuda()
b = torch.tensor(3).cuda()
c = torch.randn(3, 3).cuda()

op_with_autograd(a, b, c)

In [ ]:
import torch
import torch.utils.cpp_extension

# Load the custom module
custom_module = torch.utils.cpp_extension.load(
    name="AlexNet",
    sources=["custom_module.cpp"],
    is_python_module=False,
    verbose=True,
    build_directory="build",
    extra_cflags=["-O3"],
)

In [ ]:
dir(torch.ops.custom.CustomModule)

In [ ]:
torch.ops.

In [ ]:
# Instantiate the module
model = custom_module.CustomModule()

# test custom op serialization

In [ ]:
class Foo(nn.Module):
    def forward(self, x: Tensor, y: int, z: Tensor) -> Tensor:
        return op_with_autograd(x, y, z)


module = Foo()
module(a, b, c)

In [ ]:
scripted = jit.script(module)
scripted.save("scripted_module.pt")
scripted(a, b, c)

In [ ]:
loaded = jit.load("scripted_module.pt")
loaded(a, b, c)

## test backward

In [ ]:
import torch

torch.ops.load_library("libcustom_ops.so")

In [ ]:
torch.ops.custom.opa

In [ ]:
torch.ops.loaded_libraries

In [ ]:
print(dir(torch.ops.my_ops))
print(dir(torch.ops.custom))

In [ ]:
import torch.utils.cpp_extension

torch.utils.cpp_extension.load(
    name="warp_perspective",
    sources=["op.cpp"],
    extra_ldflags=["-lopencv_core", "-lopencv_imgproc"],
    is_python_module=False,
    verbose=True,
)

print(torch.ops.my_ops.warp_perspective)

In [ ]:
torch.ops.my_ops.op_with_autograd

In [ ]:
def f(x: Tensor) -> Tensor:
    return torch.ops.my_ops.warp_perspective(x)

In [ ]:
import torch
import torch.utils.cpp_extension

op_source = """
#include <torch/script.h>

torch::Tensor warp_perspective(torch::Tensor image, torch::Tensor warp) {
  return image.clone();
}

TORCH_LIBRARY(my_ops, m) {
  m.def("warp_perspective", &warp_perspective);
}
"""

torch.utils.cpp_extension.load_inline(
    name="warp_perspective",
    cpp_sources=op_source,
    extra_ldflags=["-lopencv_core", "-lopencv_imgproc"],
    is_python_module=False,
    verbose=True,
)

print(torch.ops.my_ops.warp_perspective)

In [ ]:
print(torch.ops.my_ops.warp_perspective)

In [ ]:
dir(torch.ops.my_ops)

In [ ]:
import torch

torch.ops.load_library("build/libwarp_perspective.so")
print(torch.ops.my_ops.warp_perspective)

In [ ]:
import torch.utils.cpp_extension

torch.utils.cpp_extension.load(
    name="warp_perspective",
    sources=["op.cpp"],
    extra_ldflags=["-lopencv_core", "-lopencv_imgproc"],
    is_python_module=False,
    verbose=True,
)

print(torch.ops.my_ops.warp_perspective)

In [ ]:
from setuptools import setup
from torch.utils.cpp_extension import BuildExtension, CppExtension

setup(
    name="warp_perspective",
    ext_modules=[
        CppExtension(
            "warp_perspective",
            ["example_app/warp_perspective/op.cpp"],
            libraries=["opencv_core", "opencv_imgproc"],
        )
    ],
    cmdclass={"build_ext": BuildExtension.with_options(no_python_abi_suffix=True)},
)

In [ ]:
def compute(x, y, z):
    x = torch.ops.my_ops.warp_perspective(x, torch.eye(3))
    return x.matmul(y) + torch.relu(z)

In [ ]:
inputs = [torch.randn(4, 8), torch.randn(8, 5), torch.randn(4, 5)]
trace = torch.jit.trace(compute, inputs)
print(trace.graph)